In [39]:
#Inserting JD and Resumes into MongoDb
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
from os import listdir
from os.path import isfile, join
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client.CaseStudy #created a db
MainCollect= db.Test #created a collection
###########
##WORKING PROPERLY: reads texts from all resume pdfs, any number of pages
###########
onlyfiles = [f for f in listdir('D:\\Case Study\\Test_Case_Study')
             if isfile(join('D:\\Case Study\\Test_Case_Study', f))]
#text = {}
for file in onlyfiles:
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    doc = join('D:\\Case Study\\Test_Case_Study', file)
    print(file)
    fhandle = open(doc, 'rb')
    for page in PDFPage.get_pages(fhandle,
                                  caching=False,
                                  check_extractable=True):
        page_interpreter.process_page(page)
    resume = fake_file_handle.getvalue()
    text = {"contents": resume,"filenames": file}
    print(resume)
    MainCollect.insert_one(text)
    fake_file_handle.close()
    converter.close()

CV_Database_Engineer_1.pdf
 

 

 

Contact

www.linkedin.com/in/matej-macek-
croatia (LinkedIn)

Matej M.

Team Lead Database Engineering | Wirecard Service Technologies
GmbH
Munich (County)

Top Skills
SQL
Database Administration
Networks

Languages
English
Croatian
German (Elementary)

Certifications
MCPS: Microsoft Certified
Professional
LFS101x.2: Introduction to Linux
Red Hat Certified Specialist in
Ansible Automation

Honors-Awards
Microsoft Virtual Academy Croatia
1st place
Cisco NetRiders 2015 Croatia 1st
place
Cisco NetRiders 2015 Europe,
Russian  Federation and CIS TOP
10
LLPA Battle 2015 Croatia TOP 5
Wirecard Innovation Challenge 2018

Summary
Experienced System Engineer (Windows, Linux) and Database
Administrator/Engineer (Microsoft SQL, DataStax Enterprise
Cassandra, MySQL, MariaDB, AWS RDS, OCI) with a demonstrated
history of working in the financial services industry in enterprise level
server environments. Strong information technology professional
with a Professional

In [40]:
#Retrieving data from MongoDB - JD & Resumes
from pymongo import MongoClient
import pandas as pd

client = MongoClient('mongodb://localhost:27017/')
with client:
    db = client.CaseStudy    
Data = db.Test.find({})
data = pd.DataFrame(Data)

In [41]:
#Contents of CV and JD
data

,_id,contents,filenames
0,5f56f3053e9545c16881c879,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Database_Engineer_1.pdf
1,5f56f3053e9545c16881c87a,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Database_Engineer_2.pdf
2,5f56f3053e9545c16881c87b,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Database_Engineer_3.pdf
3,5f56f3063e9545c16881c87c,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Data_Analyst_1.pdf
4,5f56f3063e9545c16881c87d,Contact\n\nwww.linkedin.com/in/chengkai-yao\n(...,CV_Linux_Developer_1.pdf
5,5f56f3063e9545c16881c87e,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Python_Developer_1.pdf
6,5f56f3063e9545c16881c87f,Contact\nabinabrahamcs@gmail.com\n\nwww.linked...,CV_Python_Developer_2.pdf
7,5f56f3063e9545c16881c880,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_SQL_Developer_1.pdf
8,5f56f3063e9545c16881c881,Contact\n\nwww.linkedin.com/in/mladenprajdic\n...,CV_SQL_Developer_2.pdf
9,5f56f3063e9545c16881c882,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Tableau_Developer_1.pdf


In [42]:
#Pre-processing 
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = text.replace("\n"," ")
    text = re.sub('\W+',' ', text )
    text_tokens = word_tokenize(text)
    text = [lemmatizer.lemmatize(token,pos="v") for token in text_tokens]
    tokens = [word for word in text if not word in stopwords.words()]
    text = " ".join(tokens)
    return [tokens,text]


In [43]:
#Base Model 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

def BoW(text):
    #Bag of words 
    count_vectorizer = CountVectorizer()
    BoW_features = count_vectorizer.fit_transform(text)
    return BoW_features

def TfIdf(text):
    #Tf-Idf 
    tfidf_vectorizer = TfidfVectorizer()
    TfIdf_features = tfidf_vectorizer.fit_transform(text)
    return TfIdf_features

In [44]:
#Python code to pre-process text 

t = data.contents.apply(clean_text)
data['tokens'] = [item[0] for item in t]  #Tokenization
data['pp_text'] = [item[1] for item in t] #Cleaned text

In [45]:
#Viewing preprocessed data
data

,_id,contents,filenames,tokens,pp_text
0,5f56f3053e9545c16881c879,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Database_Engineer_1.pdf,"[contact, www, linkedin, matej, macek, croatia...",contact www linkedin matej macek croatia linke...
1,5f56f3053e9545c16881c87a,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Database_Engineer_2.pdf,"[contact, www, linkedin, richard, munn, 957490...",contact www linkedin richard munn 95749033 lin...
2,5f56f3053e9545c16881c87b,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Database_Engineer_3.pdf,"[contact, www, linkedin, richclark1, linkedin,...",contact www linkedin richclark1 linkedin richa...
3,5f56f3063e9545c16881c87c,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Data_Analyst_1.pdf,"[contact, www, linkedin, alonrechavia, linkedi...",contact www linkedin alonrechavia linkedin alo...
4,5f56f3063e9545c16881c87d,Contact\n\nwww.linkedin.com/in/chengkai-yao\n(...,CV_Linux_Developer_1.pdf,"[contact, www, linkedin, chengkai, yao, linked...",contact www linkedin chengkai yao linkedin top...
5,5f56f3063e9545c16881c87e,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Python_Developer_1.pdf,"[contact, www, linkedin, clarissapodell, linke...",contact www linkedin clarissapodell linkedin c...
6,5f56f3063e9545c16881c87f,Contact\nabinabrahamcs@gmail.com\n\nwww.linked...,CV_Python_Developer_2.pdf,"[contact, abinabrahamcs, gmail, www, linkedin,...",contact abinabrahamcs gmail www linkedin abina...
7,5f56f3063e9545c16881c880,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_SQL_Developer_1.pdf,"[contact, www, linkedin, aftandil, ibrahimov, ...",contact www linkedin aftandil ibrahimov aa2199...
8,5f56f3063e9545c16881c881,Contact\n\nwww.linkedin.com/in/mladenprajdic\n...,CV_SQL_Developer_2.pdf,"[contact, www, linkedin, mladenprajdic, linked...",contact www linkedin mladenprajdic linkedin we...
9,5f56f3063e9545c16881c882,\n\n \n\n \n\nContact\n\nwww.linkedin.com/in/...,CV_Tableau_Developer_1.pdf,"[contact, www, linkedin, luigi, benigni, 80475...",contact www linkedin luigi benigni 80475313 li...


In [46]:
#BoW - All Resumes applied for database engineer position compared against JD 
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(data)-1):
    a = BoW([data.pp_text[i],data.pp_text[10]])
    val = cosine_similarity(a, a)
    print("Resume: {}     %Similarity: {}".format(data.filenames[i], val[1][0]*100))

Resume: CV_Database_Engineer_1.pdf     %Similarity: 37.08054212664648
Resume: CV_Database_Engineer_2.pdf     %Similarity: 47.76086183554883
Resume: CV_Database_Engineer_3.pdf     %Similarity: 37.08555902712474
Resume: CV_Data_Analyst_1.pdf     %Similarity: 16.55537282764405
Resume: CV_Linux_Developer_1.pdf     %Similarity: 13.833126110010763
Resume: CV_Python_Developer_1.pdf     %Similarity: 16.087904024914305
Resume: CV_Python_Developer_2.pdf     %Similarity: 30.198658737825422
Resume: CV_SQL_Developer_1.pdf     %Similarity: 18.518142939180073
Resume: CV_SQL_Developer_2.pdf     %Similarity: 14.8539961835471
Resume: CV_Tableau_Developer_1.pdf     %Similarity: 20.417867821977026


In [47]:
#Tf Idf - All Resumes applied for database engineer position compared against JD 
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(data)-1):
    a = TfIdf([data.pp_text[i],data.pp_text[10]])
    val = cosine_similarity(a, a)
    print("Resume: {}     %Similarity: {}".format(data.filenames[i], val[1][0]*100))

Resume: CV_Database_Engineer_1.pdf     %Similarity: 26.09843437032977
Resume: CV_Database_Engineer_2.pdf     %Similarity: 35.64352531777224
Resume: CV_Database_Engineer_3.pdf     %Similarity: 27.914714918088684
Resume: CV_Data_Analyst_1.pdf     %Similarity: 9.502322597401056
Resume: CV_Linux_Developer_1.pdf     %Similarity: 8.216710329176463
Resume: CV_Python_Developer_1.pdf     %Similarity: 9.563265043967824
Resume: CV_Python_Developer_2.pdf     %Similarity: 20.908197038319802
Resume: CV_SQL_Developer_1.pdf     %Similarity: 10.971667636003831
Resume: CV_SQL_Developer_2.pdf     %Similarity: 8.977759713991352
Resume: CV_Tableau_Developer_1.pdf     %Similarity: 12.27623963825996
